### Домашнее задание №1

#### Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
recommended_list=[143,156,1134,991,27,1543,3345,533,11,43,156,3345,10,15,1134]
prices_recommended_list=[400,100,400,90,60,340,70,190,110,240,120,250,90,100,450]

bought_list=[156,27,11,43,1134]
prices_bought=[100,60,110,240,400]

In [3]:
data = pd.DataFrame({"user_id": ["u1","u2","u3"], 
              "recommended_list": [[143, 156, 1134, 991, 27],
                                   [1543, 3345, 533, 11, 43],
                                   [156, 3345, 10, 15, 1134]
                                  ],
               "bought_list": [[156,27],
                               [11,43],
                               [1134]
                              ],
                "prices_recommended_list": [[400,100,400,90,60],
                                             [340,70,190,110,240],
                                             [120,250,90,100,450]
                                           ],
                 "prices_bought": [[100,60],
                                   [110,240],
                                   [400]]
                    })
data

,user_id,recommended_list,bought_list,prices_recommended_list,prices_bought
0,u1,"[143, 156, 1134, 991, 27]","[156, 27]","[400, 100, 400, 90, 60]","[100, 60]"
1,u2,"[1543, 3345, 533, 11, 43]","[11, 43]","[340, 70, 190, 110, 240]","[110, 240]"
2,u3,"[156, 3345, 10, 15, 1134]",[1134],"[120, 250, 90, 100, 450]",[400]


In [4]:
def recall_at_k(recommended_list, bought_list, k=5):
       
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
  
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommended_list, bought_list)
    
    return np.dot(flags, prices_recommended)/prices_bought.sum()
    

In [5]:
recall_at_k(recommended_list, bought_list, 5) #для топ-5 товаров

0.6

In [6]:
recall_at_k(recommended_list, bought_list, 3) # для топ-3 товаров

0.4

In [8]:
money_recall_at_k(recommended_list, bought_list, prices_recommended_list, 5) #для топ-5 товаров

112.0

In [9]:
np.round(money_recall_at_k(recommended_list, bought_list, prices_recommended_list, 3), decimals=2) # для топ-3 товаров

186.67

#### Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MRR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [10]:
recommended_list = [143,156,1134,991,27,1543,3345,533,11,43,156,3345,10,15,1134]

bought_list = [[156,27], #u1
                      [11,43],  #u2
                      [1134]]  #u3

In [11]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    amount_user = len(bought_list)
    rr = []
    for i in np.arange(amount_user):    
        relevant_indexes = np.nonzero(np.isin(recommended_list[i], bought_list[i]))[0]
        if len(relevant_indexes) != 0:
            rr.append(1/(relevant_indexes[0]+1))
    
    if len(rr) == 0:
        return 0
    
    return sum(rr)/amount_user

In [12]:
np.round(mrr_at_k(recommended_list, bought_list, 5),4) #для топ-5 товаров

C:\Users\Инна\AppData\Local\Temp\ipykernel_2984\1409222264.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bought_list = np.array(bought_list)


0.3333

In [13]:
np.round(mrr_at_k(recommended_list, bought_list, 3),decimals=4) #для топ-3 товаров

C:\Users\Инна\AppData\Local\Temp\ipykernel_2984\1409222264.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bought_list = np.array(bought_list)


0.3333

результат одинаковый

#### Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$


In [16]:
recommended_list=[143,156,1134,991,27,1543,3345,533,11,43,156,3345,10,15,1134]
prices_recommended_list=[400,100,400,90,60,340,70,190,110,240,120,250,90,100,450]

bought_list=[156,27,11,43,1134]
prices_bought=[100,60,110,240,400]

In [17]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
      
    flags = np.isin(recommended_list, bought_list)
    rank_list = []
    for i in np.arange(len(recommended_list)):
        if i < 2:
            rank_list.append(i+1)
        else:
            rank_list.append(math.log2(i+1))
    if len(recommended_list) == 0:
        return 0
    dcg = sum(np.divide(flags, rank_list)) / len(recommended_list)

    i_dcg = sum(np.divide(1, rank_list)) / len(recommended_list)

    return dcg/i_dcg

In [18]:
np.round(ndcg_at_k(recommended_list, bought_list, 5),2)

0.51